---
title: "Zig-Zag サンプラーのサブサンプリングによるスケーラビリティ"
subtitle: "大規模モデル・大規模データに対する MCMC を目指して"
author: "司馬博文"
date: 7/18/2024
date-modified: 8/29/2024
categories: [MCMC, Computation, Julia, Sampling]
image: MeanOfGaussian.svg
bibliography:
    - ../../../assets/mathematics.bib
    - ../../../assets/bib.bib
csl: ../../../assets/apalike.csl
abstract-title: 概要
abstract: Zig-Zag サンプラーは，その非対称なダイナミクスにより，収束が速くなることが期待されている MCMC 手法である．それだけでなく，対数尤度の勾配に対する不偏推定量をサブサンプリングにより構成することで，ベイズ推論においてサンプルサイズに依らない一定のコストで効率的な事後分布からのサンプリングが可能である．
code-fold: false
execute:
    cache: true
# shift-heading-level-by: -1
listing: 
    -   id: lst-listing
        type: grid
        sort: false
        contents:
            - "../Process/ZigZag.qmd"
            - "Logistic.qmd"
            - "Logistic2.qmd"
        date-format: iso
        fields: [title,image,date,subtitle]
---

::: {.hidden}

::: {.content-visible when-format="html"}

A Blog Entry on Bayesian Computation by an Applied Mathematician

$$

\renewcommand{\P}{\operatorname{P}}\newcommand{\E}{\operatorname{E}}
\newcommand{\R}{\mathbb{R}}\newcommand{\F}{\mathcal{F}}
\newcommand{\abs}[1]{\lvert#1\rvert}\newcommand{\Abs}[1]{\left|#1\right|}\newcommand{\ABs}[1]{\biggl|#1\biggr|}\newcommand{\norm}[1]{\|#1\|}\newcommand{\Norm}[1]{\left\|#1\right\|}\newcommand{\NOrm}[1]{\biggl\|#1\biggr\|}\newcommand{\Brace}[1]{\left\{#1\right\}}\newcommand{\BRace}[1]{\biggl\{#1\biggr\}}\newcommand{\paren}[1]{\left(#1\right)}\newcommand{\Paren}[1]{\biggr(#1\biggl)}\newcommand{\brac}[1]{\langle#1\rangle}\newcommand{\Brac}[1]{\left\langle#1\right\rangle}\newcommand{\BRac}[1]{\biggl\langle#1\biggr\rangle}\newcommand{\bra}[1]{\left\langle#1\right|}\newcommand{\ket}[1]{\left|#1\right\rangle}\newcommand{\Square}[1]{\left[#1\right]}\newcommand{\SQuare}[1]{\biggl[#1\biggr]}\newcommand{\rN}{\mathrm{N}}\newcommand{\ov}[1]{\overline{#1}}\newcommand{\un}[1]{\underline{#1}}\newcommand{\wt}[1]{\widetilde{#1}}\newcommand{\wh}[1]{\widehat{#1}}\newcommand{\pp}[2]{\frac{\partial #1}{\partial #2}}\newcommand{\ppp}[3]{\frac{\partial #1}{\partial #2\partial #3}}\newcommand{\dd}[2]{\frac{d #1}{d #2}}\newcommand{\floor}[1]{\lfloor#1\rfloor}\newcommand{\Floor}[1]{\left\lfloor#1\right\rfloor}\newcommand{\ceil}[1]{\lceil#1\rceil}\newcommand{\ocinterval}[1]{(#1]}\newcommand{\cointerval}[1]{[#1)}\newcommand{\COinterval}[1]{\left[#1\right)}\newcommand{\iso}{\overset{\sim}{\to}}



\newcommand{\y}{\b{y}}\newcommand{\mi}{\,|\,}\newcommand{\Mark}{\mathrm{Mark}}
\newcommand{\argmax}{\operatorname*{argmax}}\newcommand{\argmin}{\operatorname*{argmin}}

\newcommand{\pr}{\mathrm{pr}}\newcommand{\Conv}{\operatorname{Conv}}\newcommand{\cU}{\mathcal{U}}
\newcommand{\Map}{\mathrm{Map}}\newcommand{\dom}{\mathrm{Dom}\;}\newcommand{\cod}{\mathrm{Cod}\;}\newcommand{\supp}{\mathrm{supp}\;}
\newcommand{\grad}{\operatorname{grad}}\newcommand{\rot}{\operatorname{rot}}\renewcommand{\div}{\operatorname{div}}\newcommand{\tr}{\operatorname{tr}}\newcommand{\Tr}{\operatorname{Tr}}\newcommand{\KL}{\operatorname{KL}}\newcommand{\JS}{\operatorname{JS}}\newcommand{\ESS}{\operatorname{ESS}}\newcommand{\MSE}{\operatorname{MSE}}\newcommand{\erf}{\operatorname{erf}}\newcommand{\arctanh}{\operatorname{arctanh}}\newcommand{\pl}{\operatorname{pl}}\newcommand{\minimize}{\operatorname{minimize}}\newcommand{\subjectto}{\operatorname{subject to}}\newcommand{\sinc}{\operatorname{sinc}}\newcommand{\Ent}{\operatorname{Ent}}\newcommand{\Polya}{\operatorname{Polya}}\newcommand{\Exp}{\operatorname{Exp}}\newcommand{\codim}{\operatorname{codim}}\newcommand{\sgn}{\operatorname{sgn}}\newcommand{\rank}{\operatorname{rank}}

\newcommand{\vctr}[2]{\begin{pmatrix}#1\\#2\end{pmatrix}}\newcommand{\vctrr}[3]{\begin{pmatrix}#1\\#2\\#3\end{pmatrix}}\newcommand{\mtrx}[4]{\begin{pmatrix}#1&#2\\#3&#4\end{pmatrix}}\newcommand{\smtrx}[4]{\paren{\begin{smallmatrix}#1&#2\\#3&#4\end{smallmatrix}}}\newcommand{\Ker}{\mathrm{Ker}\;}\newcommand{\Coker}{\mathrm{Coker}\;}\newcommand{\Coim}{\mathrm{Coim}\;}\newcommand{\lcm}{\mathrm{lcm}}\newcommand{\GL}{\mathrm{GL}}\newcommand{\SL}{\mathrm{SL}}\newcommand{\alt}{\mathrm{alt}}

\renewcommand{\Re}{\mathrm{Re}\;}\renewcommand{\Im}{\mathrm{Im}\,}\newcommand{\Gal}{\mathrm{Gal}}\newcommand{\PGL}{\mathrm{PGL}}\newcommand{\PSL}{\mathrm{PSL}}\newcommand{\Log}{\mathrm{Log}\,}\newcommand{\Res}{\mathrm{Res}\,}\newcommand{\on}{\mathrm{on}\;}\newcommand{\hatC}{\widehat{\C}}\newcommand{\hatR}{\hat{\R}}\newcommand{\PV}{\mathrm{P.V.}}\newcommand{\diam}{\mathrm{diam}}\newcommand{\Area}{\mathrm{Area}}\newcommand{\Lap}{\Laplace}\newcommand{\f}{\mathbf{f}}\newcommand{\cR}{\mathcal{R}}\newcommand{\const}{\mathrm{const.}}\newcommand{\Om}{\Omega}\newcommand{\Cinf}{C^\infty}\newcommand{\ep}{\epsilon}\newcommand{\dist}{\mathrm{dist}}\newcommand{\opart}{\o{\partial}}\newcommand{\Length}{\mathrm{Length}}

\newcommand{\cA}{\mathcal{A}}\newcommand{\cO}{\mathcal{O}}\newcommand{\cW}{\mathcal{W}}\renewcommand{\O}{\mathcal{O}}\renewcommand{\S}{\mathcal{S}}\newcommand{\U}{\mathcal{U}}\newcommand{\V}{\mathrm{V}}\newcommand{\N}{\mathbb{N}}\newcommand{\bN}{\mathbb{N}}\newcommand{\C}{\mathrm{C}}\newcommand{\bC}{\mathbb{C}}\newcommand{\Z}{\mathcal{Z}}\newcommand{\Q}{\mathbb{Q}}\newcommand{\bQ}{\mathbb{Q}}\newcommand{\TV}{\mathrm{TV}}\newcommand{\ORD}{\mathrm{ORD}}\newcommand{\Card}{\mathrm{Card}\,}\newcommand{\Top}{\mathrm{Top}}\newcommand{\Disc}{\mathrm{Disc}}\newcommand{\Codisc}{\mathrm{Codisc}}\newcommand{\CoDisc}{\mathrm{CoDisc}}\newcommand{\Ult}{\mathrm{Ult}}\newcommand{\ord}{\mathrm{ord}}\newcommand{\bS}{\mathbb{S}}\newcommand{\PConn}{\mathrm{PConn}}\newcommand{\mult}{\mathrm{mult}}\newcommand{\inv}{\mathrm{inv}}

\newcommand{\Der}{\mathrm{Der}}\newcommand{\osub}{\overset{\mathrm{open}}{\subset}}\newcommand{\osup}{\overset{\mathrm{open}}{\supset}}\newcommand{\al}{\alpha}\newcommand{\K}{\mathbb{K}}\newcommand{\Sp}{\mathrm{Sp}}\newcommand{\g}{\mathfrak{g}}\newcommand{\h}{\mathfrak{h}}\newcommand{\Imm}{\mathrm{Imm}}\newcommand{\Imb}{\mathrm{Imb}}\newcommand{\Gr}{\mathrm{Gr}}

\newcommand{\Ad}{\mathrm{Ad}}\newcommand{\finsupp}{\mathrm{fin\;supp}}\newcommand{\SO}{\mathrm{SO}}\newcommand{\SU}{\mathrm{SU}}\newcommand{\acts}{\curvearrowright}\newcommand{\mono}{\hookrightarrow}\newcommand{\epi}{\twoheadrightarrow}\newcommand{\Stab}{\mathrm{Stab}}\newcommand{\nor}{\mathrm{nor}}\newcommand{\T}{\mathbb{T}}\newcommand{\Aff}{\mathrm{Aff}}\newcommand{\rsup}{\triangleright}\newcommand{\subgrp}{\overset{\mathrm{subgrp}}{\subset}}\newcommand{\Ext}{\mathrm{Ext}}\newcommand{\sbs}{\subset}\newcommand{\sps}{\supset}\newcommand{\In}{\mathrm{in}\;}\newcommand{\Tor}{\mathrm{Tor}}\newcommand{\p}{\b{p}}\newcommand{\q}{\mathfrak{q}}\newcommand{\m}{\mathfrak{m}}\newcommand{\cS}{\mathcal{S}}\newcommand{\Frac}{\mathrm{Frac}\,}\newcommand{\Spec}{\mathrm{Spec}\,}\newcommand{\bA}{\mathbb{A}}\newcommand{\Sym}{\mathrm{Sym}}\newcommand{\Ann}{\mathrm{Ann}}\newcommand{\Her}{\mathrm{Her}}\newcommand{\Bil}{\mathrm{Bil}}\newcommand{\Ses}{\mathrm{Ses}}\newcommand{\FVS}{\mathrm{FVS}}

\newcommand{\Ho}{\mathrm{Ho}}\newcommand{\CW}{\mathrm{CW}}\newcommand{\lc}{\mathrm{lc}}\newcommand{\cg}{\mathrm{cg}}\newcommand{\Fib}{\mathrm{Fib}}\newcommand{\Cyl}{\mathrm{Cyl}}\newcommand{\Ch}{\mathrm{Ch}}
\newcommand{\rP}{\mathrm{P}}\newcommand{\rE}{\mathrm{E}}\newcommand{\e}{\b{e}}\renewcommand{\k}{\b{k}}\newcommand{\Christ}[2]{\begin{Bmatrix}#1\\#2\end{Bmatrix}}\renewcommand{\Vec}[1]{\overrightarrow{\mathrm{#1}}}\newcommand{\hen}[1]{\mathrm{#1}}\renewcommand{\b}[1]{\boldsymbol{#1}}

\newcommand{\Inc}{\mathrm{Inc}}\newcommand{\aInc}{\mathrm{aInc}}\newcommand{\HS}{\mathrm{HS}}\newcommand{\loc}{\mathrm{loc}}\newcommand{\Lh}{\mathrm{L.h.}}\newcommand{\Epi}{\mathrm{Epi}}\newcommand{\slim}{\mathrm{slim}}\newcommand{\Ban}{\mathrm{Ban}}\newcommand{\Hilb}{\mathrm{Hilb}}\newcommand{\Ex}{\mathrm{Ex}}\newcommand{\Co}{\mathrm{Co}}\newcommand{\sa}{\mathrm{sa}}\newcommand{\nnorm}[1]{{\left\vert\kern-0.25ex\left\vert\kern-0.25ex\left\vert #1 \right\vert\kern-0.25ex\right\vert\kern-0.25ex\right\vert}}\newcommand{\dvol}{\mathrm{dvol}}\newcommand{\Sconv}{\mathrm{Sconv}}\newcommand{\I}{\mathcal{I}}\newcommand{\nonunital}{\mathrm{nu}}\newcommand{\cpt}{\mathrm{cpt}}\newcommand{\lcpt}{\mathrm{lcpt}}\newcommand{\com}{\mathrm{com}}\newcommand{\Haus}{\mathrm{Haus}}\newcommand{\proper}{\mathrm{proper}}\newcommand{\infinity}{\mathrm{inf}}\newcommand{\TVS}{\mathrm{TVS}}\newcommand{\ess}{\mathrm{ess}}\newcommand{\ext}{\mathrm{ext}}\newcommand{\Index}{\mathrm{Index}\;}\newcommand{\SSR}{\mathrm{SSR}}\newcommand{\vs}{\mathrm{vs.}}\newcommand{\fM}{\mathfrak{M}}\newcommand{\EDM}{\mathrm{EDM}}\newcommand{\Tw}{\mathrm{Tw}}\newcommand{\fC}{\mathfrak{C}}\newcommand{\bn}{\boldsymbol{n}}\newcommand{\br}{\boldsymbol{r}}\newcommand{\Lam}{\Lambda}\newcommand{\lam}{\lambda}\newcommand{\one}{\mathbf{1}}\newcommand{\dae}{\text{-a.e.}}\newcommand{\das}{\text{-a.s.}}\newcommand{\td}{\text{-}}\newcommand{\RM}{\mathrm{RM}}\newcommand{\BV}{\mathrm{BV}}\newcommand{\normal}{\mathrm{normal}}\newcommand{\lub}{\mathrm{lub}\;}\newcommand{\Graph}{\mathrm{Graph}}\newcommand{\Ascent}{\mathrm{Ascent}}\newcommand{\Descent}{\mathrm{Descent}}\newcommand{\BIL}{\mathrm{BIL}}\newcommand{\fL}{\mathfrak{L}}\newcommand{\De}{\Delta}

\newcommand{\calA}{\mathcal{A}}\newcommand{\calB}{\mathcal{B}}\newcommand{\D}{\mathcal{D}}\newcommand{\Y}{\mathcal{Y}}\newcommand{\calC}{\mathcal{C}}\renewcommand{\ae}{\mathrm{a.e.}\;}\newcommand{\cZ}{\mathcal{Z}}\newcommand{\fF}{\mathfrak{F}}\newcommand{\fI}{\mathfrak{I}}\newcommand{\rV}{\mathrm{V}}\newcommand{\cE}{\mathcal{E}}\newcommand{\sMap}{\sigma\textrm{-}\mathrm{Map}}\newcommand{\cC}{\mathcal{C}}\newcommand{\comp}{\complement}\newcommand{\J}{\mathcal{J}}\newcommand{\sumN}[1]{\sum_{#1\in\N}}\newcommand{\cupN}[1]{\cup_{#1\in\N}}\newcommand{\capN}[1]{\cap_{#1\in\N}}\newcommand{\Sum}[1]{\sum_{#1=1}^\infty}\newcommand{\sumn}{\sum_{n=1}^\infty}\newcommand{\summ}{\sum_{m=1}^\infty}\newcommand{\sumk}{\sum_{k=1}^\infty}\newcommand{\sumi}{\sum_{i=1}^\infty}\newcommand{\sumj}{\sum_{j=1}^\infty}\newcommand{\cupn}{\cup_{n=1}^\infty}\newcommand{\capn}{\cap_{n=1}^\infty}\newcommand{\cupk}{\cup_{k=1}^\infty}\newcommand{\cupi}{\cup_{i=1}^\infty}\newcommand{\cupj}{\cup_{j=1}^\infty}\newcommand{\limn}{\lim_{n\to\infty}}\renewcommand{\L}{\mathcal{L}}\newcommand{\cL}{\mathcal{L}}\newcommand{\Cl}{\mathrm{Cl}}\newcommand{\cN}{\mathcal{N}}\newcommand{\Ae}{\textrm{-a.e.}\;}\renewcommand{\csub}{\overset{\textrm{closed}}{\subset}}\renewcommand{\csup}{\overset{\textrm{closed}}{\supset}}\newcommand{\wB}{\wt{B}}\newcommand{\cG}{\mathcal{G}}\newcommand{\Lip}{\mathrm{Lip}}\newcommand{\AC}{\mathrm{AC}}\newcommand{\Mol}{\mathrm{Mol}}

\newcommand{\Pe}{\mathrm{Pe}}\newcommand{\wR}{\wh{\mathbb{\R}}}\newcommand*{\Laplace}{\mathop{}\!\mathbin\bigtriangleup}\newcommand*{\DAlambert}{\mathop{}\!\mathbin\Box}\newcommand{\bT}{\mathbb{T}}\newcommand{\dx}{\dslash x}\newcommand{\dt}{\dslash t}\newcommand{\ds}{\dslash s}

\newcommand{\round}{\mathrm{round}}\newcommand{\cond}{\mathrm{cond}}\newcommand{\diag}{\mathrm{diag}}
\newcommand{\Adj}{\mathrm{Adj}}\newcommand{\Pf}{\mathrm{Pf}}\newcommand{\Sg}{\mathrm{Sg}}


\newcommand{\aseq}{\overset{\text{a.s.}}{=}}\newcommand{\deq}{\overset{\text{d}}{=}}\newcommand{\cV}{\mathcal{V}}\newcommand{\FM}{\mathrm{FM}}\newcommand{\KR}{\mathrm{KR}}\newcommand{\rba}{\mathrm{rba}}\newcommand{\rca}{\mathrm{rca}}\newcommand{\Prob}{\mathrm{Prob}}\newcommand{\X}{\mathcal{X}}\newcommand{\Meas}{\mathrm{Meas}}\newcommand{\as}{\;\text{a.s.}}\newcommand{\io}{\;\mathrm{i.o.}}\newcommand{\fe}{\;\text{f.e.}}\newcommand{\bF}{\mathbb{F}}\newcommand{\W}{\mathcal{W}}\newcommand{\Pois}{\mathrm{Pois}}\newcommand{\iid}{\text{i.i.d.}}\newcommand{\wconv}{\rightsquigarrow}\newcommand{\Var}{\mathrm{Var}}\newcommand{\xrightarrown}{\xrightarrow{n\to\infty}}\newcommand{\au}{\mathrm{au}}\newcommand{\cT}{\mathcal{T}}\newcommand{\wto}{\overset{\text{w}}{\to}}\newcommand{\dto}{\overset{\text{d}}{\to}}\newcommand{\sto}{\overset{\text{s}}{\to}}\newcommand{\pto}{\overset{\text{p}}{\to}}\newcommand{\mto}{\overset{\text{m}}{\to}}\newcommand{\vto}{\overset{v}{\to}}\newcommand{\Cont}{\mathrm{Cont}}\newcommand{\stably}{\mathrm{stably}}\newcommand{\Np}{\mathbb{N}^+}\newcommand{\oM}{\overline{\mathcal{M}}}\newcommand{\fP}{\mathfrak{P}}\newcommand{\sign}{\mathrm{sign}}
\newcommand{\Borel}{\mathrm{Borel}}\newcommand{\Mid}{\,|\,}\newcommand{\middleMid}{\;\middle|\;}\newcommand{\CP}{\mathrm{CP}}\newcommand{\bD}{\mathbb{D}}\newcommand{\bL}{\mathbb{L}}\newcommand{\fW}{\mathfrak{W}}\newcommand{\DL}{\mathcal{D}\mathcal{L}}\renewcommand{\r}[1]{\mathrm{#1}}\newcommand{\rC}{\mathrm{C}}\newcommand{\qqquad}{\qquad\quad}

\newcommand{\bit}{\mathrm{bit}}

\newcommand{\err}{\mathrm{err}}

\newcommand{\varparallel}{\mathbin{\!/\mkern-5mu/\!}}\newcommand{\Ri}{\mathrm{Ri}}\newcommand{\Cone}{\mathrm{Cone}}\newcommand{\Int}{\mathrm{Int}}

\newcommand{\pre}{\mathrm{pre}}\newcommand{\om}{\omega}


\newcommand{\del}{\partial}
\newcommand{\LHS}{\mathrm{LHS}}\newcommand{\RHS}{\mathrm{RHS}}\newcommand{\bnu}{\boldsymbol{\nu}}\newcommand{\interior}{\mathrm{in}\;}\newcommand{\SH}{\mathrm{SH}}\renewcommand{\v}{\boldsymbol{\nu}}\newcommand{\n}{\mathbf{n}}\newcommand{\ssub}{\Subset}\newcommand{\curl}{\mathrm{curl}}

\newcommand{\Ei}{\mathrm{Ei}}\newcommand{\sn}{\mathrm{sn}}\newcommand{\wgamma}{\widetilde{\gamma}}

\newcommand{\Ens}{\mathrm{Ens}}

\newcommand{\cl}{\mathrm{cl}}\newcommand{\x}{\boldsymbol{x}}

\newcommand{\Do}{\mathrm{Do}}\newcommand{\IV}{\mathrm{IV}}

\newcommand{\AIC}{\mathrm{AIC}}\newcommand{\mrl}{\mathrm{mrl}}\newcommand{\dotx}{\dot{x}}\newcommand{\UMV}{\mathrm{UMV}}\newcommand{\BLU}{\mathrm{BLU}}

\newcommand{\comb}[2]{\begin{pmatrix}#1\\#2\end{pmatrix}}\newcommand{\bP}{\mathbb{P}}\newcommand{\compsub}{\overset{\textrm{cpt}}{\subset}}\newcommand{\lip}{\textrm{lip}}\newcommand{\BL}{\mathrm{BL}}\newcommand{\G}{\mathbb{G}}\newcommand{\NB}{\mathrm{NB}}\newcommand{\oR}{\ov{\R}}\newcommand{\liminfn}{\liminf_{n\to\infty}}\newcommand{\limsupn}{\limsup_{n\to\infty}}\newcommand{\esssup}{\mathrm{ess.sup}}\newcommand{\asto}{\xrightarrow{\as}}\newcommand{\Cov}{\mathrm{Cov}}\newcommand{\cQ}{\mathcal{Q}}\newcommand{\VC}{\mathrm{VC}}\newcommand{\mb}{\mathrm{mb}}\newcommand{\Avar}{\mathrm{Avar}}\newcommand{\bB}{\mathbb{B}}\newcommand{\bW}{\mathbb{W}}\newcommand{\sd}{\mathrm{sd}}\newcommand{\w}[1]{\widehat{#1}}\newcommand{\bZ}{\mathbb{Z}}\newcommand{\Bernoulli}{\mathrm{Ber}}\newcommand{\Ber}{\mathrm{Ber}}\newcommand{\Mult}{\mathrm{Mult}}\newcommand{\BPois}{\mathrm{BPois}}\newcommand{\fraks}{\mathfrak{s}}\newcommand{\frakk}{\mathfrak{k}}\newcommand{\IF}{\mathrm{IF}}\newcommand{\bX}{\boldsymbol{X}}\newcommand{\bx}{\boldsymbol{x}}\newcommand{\indep}{\perp\!\!\!\perp}\newcommand{\IG}{\mathrm{IG}}\newcommand{\Levy}{\mathrm{Levy}}\newcommand{\MP}{\mathrm{MP}}\newcommand{\Hermite}{\mathrm{Hermite}}\newcommand{\Skellam}{\mathrm{Skellam}}\newcommand{\Dirichlet}{\mathrm{Dirichlet}}\renewcommand{\Beta}{\operatorname{Beta}}\newcommand{\bE}{\mathbb{E}}\newcommand{\bG}{\mathbb{G}}\newcommand{\MISE}{\mathrm{MISE}}\newcommand{\logit}{\mathtt{logit}}\newcommand{\expit}{\mathtt{expit}}\newcommand{\cK}{\mathcal{K}}\newcommand{\dl}{\dot{l}}\newcommand{\dotp}{\dot{p}}\newcommand{\wl}{\wt{l}}\newcommand{\Gauss}{\mathrm{Gauss}}\newcommand{\fA}{\mathfrak{A}}\newcommand{\under}{\mathrm{under}\;}\newcommand{\whtheta}{\wh{\theta}}\newcommand{\Em}{\mathrm{Em}}\newcommand{\ztheta}{{\theta_0}}
\newcommand{\rO}{\mathrm{O}}\newcommand{\Bin}{\mathrm{Bin}}\newcommand{\rW}{\mathrm{W}}\newcommand{\rG}{\mathrm{G}}\newcommand{\rB}{\mathrm{B}}\newcommand{\rU}{\mathrm{U}}\newcommand{\HG}{\mathrm{HG}}\newcommand{\GAMMA}{\mathrm{Gamma}}\newcommand{\Cauchy}{\mathrm{Cauchy}}\newcommand{\rt}{\mathrm{t}}\newcommand{\rF}{\mathrm{F}}
\newcommand{\FE}{\mathrm{FE}}\newcommand{\bV}{\boldsymbol{V}}\newcommand{\GLS}{\mathrm{GLS}}\newcommand{\be}{\boldsymbol{e}}\newcommand{\POOL}{\mathrm{POOL}}\newcommand{\GMM}{\mathrm{GMM}}\newcommand{\MM}{\mathrm{MM}}\newcommand{\SSIV}{\mathrm{SSIV}}\newcommand{\JIV}{\mathrm{JIV}}\newcommand{\AR}{\mathrm{AR}}\newcommand{\ILS}{\mathrm{ILS}}\newcommand{\SLS}{\mathrm{SLS}}\newcommand{\LIML}{\mathrm{LIML}}

\newcommand{\Rad}{\mathrm{Rad}}\newcommand{\bY}{\boldsymbol{Y}}\newcommand{\pone}{{(1)}}\newcommand{\ptwo}{{(2)}}\newcommand{\ps}[1]{{(#1)}}\newcommand{\fsub}{\overset{\text{finite}}{\subset}}


\newcommand{\varlim}{\varprojlim}\newcommand{\Hom}{\mathrm{Hom}}\newcommand{\Iso}{\mathrm{Iso}}\newcommand{\Mor}{\mathrm{Mor}}\newcommand{\Isom}{\mathrm{Isom}}\newcommand{\Aut}{\mathrm{Aut}}\newcommand{\End}{\mathrm{End}}\newcommand{\op}{\mathrm{op}}\newcommand{\ev}{\mathrm{ev}}\newcommand{\Ob}{\mathrm{Ob}}\newcommand{\Ar}{\mathrm{Ar}}\newcommand{\Arr}{\mathrm{Arr}}\newcommand{\Set}{\mathrm{Set}}\newcommand{\Grp}{\mathrm{Grp}}\newcommand{\Cat}{\mathrm{Cat}}\newcommand{\Mon}{\mathrm{Mon}}\newcommand{\Ring}{\mathrm{Ring}}\newcommand{\CRing}{\mathrm{CRing}}\newcommand{\Ab}{\mathrm{Ab}}\newcommand{\Pos}{\mathrm{Pos}}\newcommand{\Vect}{\mathrm{Vect}}\newcommand{\FinVect}{\mathrm{FinVect}}\newcommand{\FinSet}{\mathrm{FinSet}}\newcommand{\FinMeas}{\mathrm{FinMeas}}\newcommand{\OmegaAlg}{\Omega\text{-}\mathrm{Alg}}\newcommand{\OmegaEAlg}{(\Omega,E)\text{-}\mathrm{Alg}}\newcommand{\Fun}{\mathrm{Fun}}\newcommand{\Func}{\mathrm{Func}}

\newcommand{\Stoch}{\mathrm{Stoch}}\newcommand{\FinStoch}{\mathrm{FinStoch}}\newcommand{\Copy}{\mathrm{copy}}\newcommand{\Delete}{\mathrm{delete}}
\newcommand{\Bool}{\mathrm{Bool}}\newcommand{\CABool}{\mathrm{CABool}}\newcommand{\CompBoolAlg}{\mathrm{CompBoolAlg}}\newcommand{\BoolAlg}{\mathrm{BoolAlg}}\newcommand{\BoolRng}{\mathrm{BoolRng}}\newcommand{\HeytAlg}{\mathrm{HeytAlg}}\newcommand{\CompHeytAlg}{\mathrm{CompHeytAlg}}\newcommand{\Lat}{\mathrm{Lat}}\newcommand{\CompLat}{\mathrm{CompLat}}\newcommand{\SemiLat}{\mathrm{SemiLat}}\newcommand{\Stone}{\mathrm{Stone}}\newcommand{\Mfd}{\mathrm{Mfd}}\newcommand{\LieAlg}{\mathrm{LieAlg}}
\newcommand{\Op}{\mathrm{Op}}
\newcommand{\Sh}{\mathrm{Sh}}
\newcommand{\Diff}{\mathrm{Diff}}
\newcommand{\B}{\mathcal{B}}\newcommand{\cB}{\mathcal{B}}\newcommand{\Span}{\mathrm{Span}}\newcommand{\Corr}{\mathrm{Corr}}\newcommand{\Decat}{\mathrm{Decat}}\newcommand{\Rep}{\mathrm{Rep}}\newcommand{\Grpd}{\mathrm{Grpd}}\newcommand{\sSet}{\mathrm{sSet}}\newcommand{\Mod}{\mathrm{Mod}}\newcommand{\SmoothMnf}{\mathrm{SmoothMnf}}\newcommand{\coker}{\mathrm{coker}}\newcommand{\Ord}{\mathrm{Ord}}\newcommand{\eq}{\mathrm{eq}}\newcommand{\coeq}{\mathrm{coeq}}\newcommand{\act}{\mathrm{act}}

\newcommand{\apf}{\mathrm{apf}}\newcommand{\opt}{\mathrm{opt}}\newcommand{\IS}{\mathrm{IS}}\newcommand{\IR}{\mathrm{IR}}\newcommand{\iidsim}{\overset{\text{i.i.d.}}{\sim}}\newcommand{\propt}{\,\propto\,}\newcommand{\bM}{\mathbb{M}}\newcommand{\cX}{\mathcal{X}}\newcommand{\cY}{\mathcal{Y}}\newcommand{\cP}{\mathcal{P}}\newcommand{\ola}[1]{\overleftarrow{#1}}

\renewcommand{\iff}{\;\mathrm{iff}\;}
\newcommand{\False}{\mathrm{False}}\newcommand{\True}{\mathrm{True}}
\newcommand{\otherwise}{\mathrm{otherwise}}
\newcommand{\suchthat}{\;\mathrm{s.t.}\;}

\newcommand{\cM}{\mathcal{M}}\newcommand{\M}{\mathbb{M}}\newcommand{\cF}{\mathcal{F}}\newcommand{\cD}{\mathcal{D}}\newcommand{\fX}{\mathfrak{X}}\newcommand{\fY}{\mathfrak{Y}}\newcommand{\fZ}{\mathfrak{Z}}\renewcommand{\H}{\mathcal{H}}\newcommand{\cH}{\mathcal{H}}\newcommand{\fH}{\mathfrak{H}}\newcommand{\bH}{\mathbb{H}}\newcommand{\id}{\mathrm{id}}\newcommand{\A}{\mathcal{A}}
\newcommand{\lmd}{\lambda}
\newcommand{\Lmd}{\Lambda}
\newcommand{\cI}{\mathcal{I}}

\newcommand{\Lrarrow}{\;\;\Leftrightarrow\;\;}
\DeclareMathOperator{\des}{des}
\DeclareMathOperator{\nd}{nd}
\DeclareMathOperator{\dsep}{d-sep}
\DeclareMathOperator{\sep}{sep}
\newcommand{\rLL}{\mathrm{LL}}\newcommand{\HT}{\mathrm{HT}}\newcommand{\PS}{\mathrm{PS}}\newcommand{\rI}{\mathrm{I}}
$$

:::

:::



::: {#lst-listing}
:::

<!-- ## Subsampling Technology -->

MCMC の計算複雑性のボトルネックは，尤度の評価にある．各ステップで全てのデータを用いて尤度を計算する必要がある点が，MCMC を深層学習などの大規模データの設定への応用を難しくしている [@Murphy2023 p.647]．

サブサンプリングが可能であることと，複数の効率的なサブサンプリング法の提案により，Zig-Zag 過程は次世代のサンプラーとして圧倒的なスケーラビリティ（Super Efficient Bayesian Inference [@Bierkens+2019]）を示すのではないかと期待されている．^[この２点が両方肝心である．効率的なサブサンプリング推定量の開発が [@Fearnhead+2018-PDMC] 以来議論の焦点になっている．]

## 対数尤度の勾配を不変推定する

$p(x)$ を事前分布，$p(y|x)$ を観測のモデル（または尤度）とし，データ $y_1,\cdots,y_n$ は互いに独立であるとする．

このとき，事後分布 $\pi(x):=p(x|y)$ と Hamiltonian $U$ は次のように表せる：
$$
\pi(x)\propt\paren{\prod_{k=1}^n p(y_k|x)}p(x)
$$
\begin{align*}
   U(x)&=-\sum_{k=1}^n\log p(y_k|x)-\log p(x)\\
   &=\frac{1}{n}\sum_{k=1}^n\Paren{-n\log p(y_k|x)-\log p(x)}=:\frac{1}{n}\sum_{k=1}^nU^k(x).
\end{align*}

このとき，$U$ の導関数 $\partial_i U(x)$ は，独立な観測 $y_1,\cdots,y_n$ について項別微分をして平均をとったものに等しい：
$$
\partial_iU(x)=\frac{1}{n}\sum_{k=1}^nE^k_i(x),
$$ {#eq-decomposition-of-DU}
$$
E^k_i(x):=\partial_iU^k(x)=\pp{}{x_i}\Paren{-n\log p(y_k|x)-\log p(x)}.
$$

よって，精度は劣るかもしれないが，一様に選んだ $K\sim\rU([n])$ から定まる $E^K_i$ の値は $\partial_i U(x)$ の不偏推定量となっている．この発想により，ZZ-SS という新たなアルゴリズムを構成できる．

## サブサンプリングを取り入れた Zig-Zag サンプラー

この各 $E^K_i$ が定める強度関数
$$
m^K_i(t):=\Paren{\theta E^K_i(x+\theta t)}_+=\Paren{\theta\partial_iU^K(x+\theta t)}_+
$$
を用いた Zig-Zag サンプラーを [@Bierkens+2019] では **ZZ-SS** (Zig-Zag with Sub-Sampling) と呼んでいる．

$$
\max_{k\in[n]}m^k_i\le M_i
$$
を満たす連続関数 $M_i$ を用いて次のようにシミュレーションすることができる：

::: {.callout-tip appearance="simple" icon="false" title="[@Bierkens+2019 p.1303 アルゴリズム３]"}

1. 代理強度関数 $M_1,\cdots,M_d$ を持つ互いに独立な $\R_+$ 上の非一様 Poisson 点過程の到着時刻 $T_1,\cdots,T_d$ をシミュレーションする．
2. 最初に到着した座標番号 $j:=\argmin_{i\in[d]}T_i$ について，確率
    $$
    \frac{m^K_j(T_j)}{M_j(T_j)},\qquad K\sim\rU([n]),
    $$
    で時刻 $T_j$ に速度成分 $\theta_j$ の符号を反転させる．
3. １に $t=T_j$ として戻って，繰り返す．

:::

::: {.callout-note title="部分サンプリングにより不変分布が変わらないことの証明^[[@Vasdekis2021 p.25] や [@Bierkens+2019 p.1302 定理4.1] も参照．]" icon="false" collapse="true"}

ZZ-SS によってシミュレートされる過程は，レート関数
$$
\lambda_i(x,\theta)=\E[(\theta E^K_i(x))_+]=\frac{1}{n}\sum_{k=1}^n(\theta E^k_i(x))_+
$$
を持った Zig-Zag 過程に等しい

これは，元々のレート関数に対して，
$$
\gamma_i(x,\theta):=\frac{1}{n}\sum_{k=1}^n(\theta_iE^k_i(x))_+-\paren{\theta_i\frac{1}{n}\sum_{k=1}^nE^k_i(x)}_+
$$
という項を加えて得る Zig-Zag サンプラーともみなすことができる．非負性は関数 $(x)_+:=x\lor0$ の凸性から従う．最後に $\gamma_i(x,\theta)=\gamma_i(x,F_i(\theta))$ を確認すれば良い．

これは
\begin{align*}
  &\qquad\frac{1}{n}\sum_{k=1}^n\Paren{\theta_iE_i^k(x)}_+-\frac{1}{n}\sum_{k=1}^n\Paren{-\theta_iE_i^k(x)}_+\\
  &=\frac{1}{n}\sum_{k=1}^n\paren{(\theta_iE_i^k(x))_+-(-\theta_iE_i^k(x))_+}=\frac{1}{n}\sum_{k=1}^n\theta_iE_i^k(x)
\end{align*}
であることから従う．

こうして，サブサンプリングの実行による精度の劣化が，[@Andrieu-Livingstone2021] の枠組みで捉えられる，ということでもある（レート関数が増加したので，スイッチングイベントが増え，diffusive な動きが増加する）．

:::

例えば $p(y_k|x)$ が Cauchy 密度であるなど $\partial_iU$ が有界であるとき，$M_i:=\max_{x\in\R^d}\partial_iU(x)$ などと選ぶことができる．$M_i$ をより $\partial_iU$ に近く選ぶほど剪定の効率は上がるが，$M_i$ を複雑にしすぎると今度は $M_i$ を強度とする Poisson 点過程のシミュレーションが困難になる．

そのため，ZZ-SS では代理レート関数 $M_i$ は大きく取る必要があり，尤度関数の評価の回数が増える．そのため，アルゴリズムの計算複雑性は上がっていることに注意 [@Bierkens+2019 p.1302 第４節]．

## 制御変数による分散低減 {#sec-ZZ-CV}

$\partial_iU(x)$ が Lipschitz 連続であるとき，$E_i^k$ をある参照点 $\partial_iU(x_*)$ とそこからの乖離と取ることで $n\to\infty$ の極限で分散が抑えられる．

こうすることで，$M_i$ を１次関数としたまま，より小さく $E_i^k$ にフィットするように取ることができる．

::: {.callout-tip title="命題" icon="false"}

任意の $i\in[d]$ について，ある $C_i>0$ が存在して，
$$
\abs{\partial_iU(x)-\partial_iU(y)}\le C_i\abs{x-y},\quad(x,y)\in\R^{2d},
$$
が成り立つとする．このとき，
$$
M_i(t):=a_i+b_it
$$
$$
a_i:=(\theta_i\partial_iU(x_*))_++C_i\norm{x-x_*}_p,\quad b_i:=C_id^{1/p}
$$
と定めれば，
$$
m_i^k\le M_i
$$
が成り立つ．ただし，
$$
\partial_iU(x)=\frac{1}{n}\sum_{k=1}^nE^k_i(x),\tag{1}
$$
$$
E^k_i(x):=\partial_iU(x_*)+\partial_iU^k(x)-\partial_iU^k(x_*),
$$
$$
m^k_i(t):=\Paren{\theta E_i^k(x+\theta t)}_+,
$$
とした．

:::

この仮定は例えば $\partial_iU$ が有界な導関数を持つならば成り立つ．$p(y_k|x)$ が Gauss 密度であるやさらに裾が重いときは成り立つ．

次のようにして参照点 $x_*$ を選ぶ事前処理を行うことで，**データのサイズに依存しない計算複雑性で事後分布からの正確なサンプリングが可能**になる．

::: {.callout-tip appearance="simple" icon="false" title="preprocessing for ZZ-CV"}

1. $x_*:=\argmin_{x\in\R^d}U(x)$ を探索する．
2. $\partial_iU(x_*),\partial_iU^k(x_*)$ を計算する．

この２つはいずれも $O(n)$ の複雑性で実行できる．

:::

## ZZ-CV のスケーリング

このとき，$x_*$ を定める事前処理が，$\wh{x}$ を最尤推定量として，
$$
\norm{x_*-\wh{x}}_p=O(n^{-1/2})\quad(n\to\infty)
$$
程度の正確性があれば，事後分布の最尤推定量周りの漸近展開 [@Johnson1970] を通じて，
$$
\norm{x-x_*}_p=O_p(n^{-1/2})\quad(n\to\infty)
$$
$$
\partial_iU(x_*)=O_p(n^{1/2})\quad(n\to\infty)
$$
が成り立つ．

::: {.callout-important collapse="true" title="Zig-Zag 過程のスケーリング^[[@Bierkens+2019 pp.1306-] 第5.1節参照．]" icon="false"}

事後分布に対する Zig-Zag 過程は，$\sqrt{n}$ だけ時間を加速したものが $\rN_d(0,i(x_0))$ を標的にする Zig-Zag 過程に収束するから，$O(n^{-1/2})$ のタイムステップで区切ってサンプルとすることができる．

しかし
$$
\max_{k\in[n]}\Paren{\theta_i\partial_iU^k(x+\theta t)}_+\le M_i
$$
を満たす $M_i$ は $O(n^{\al})\;(\al\ge1/2)$ のスケールで増大していく．

各スイッチングイベントにおいて，全データにアクセスする $O(n)$ の計算複雑性が必要であるから，総じて $O(n^{\al+1/2})$ の計算複雑性となる． 

:::

ZZ-CV が平衡に至っている場合は $x$ はほとんど $x_*$ に集積するため，
$$
\abs{E^k_i(x)}=\ABs{\partial_iU(x_*)+\partial_iU^k(x)-\partial_iU^k(x_*)}=O(n^{1/2})
$$
が成り立つ．よってこれを抑える $M_i$ も $O(n^{1/2})$ で済み，必要以上に大きい代理レート関数を用意して剪定する必要がない．

全データにアクセスする $O(n)$ のステップもないために，事前処理 [-@sec-ZZ-CV] と十分平衡に至っているとみなせるまでの burn-in を除いて，$O(1)$ の計算複雑性でサンプリングが可能である．このことを [@Bierkens+2019] は super-efficiency と呼ぶ．

::: {.callout-important title="更なるスケーラブル手法の可能性（事後分布が集中する場合のみ？）" collapse="true" icon="false"}

他に，事後分布の集中領域でうまくスイッチング回数が抑えられる $\lambda_i$ が構成できたならば，低い計算複雑性を達成できるだろう．

ZZ-CV では，これに事後分布の Gauss 近似を用いたことになる．

また，$U$ の２階微分が有界でない場合，この枠組みが使えない．実際，[@Bierkens+2019 p.1315 第6.5節] ではこの場合での数値実験の結果が示されており，事後分布が集積しないために super-efficiency は得られていない．

参照点 $x_*$ を複数取る拡張なども [@Bierkens+2019 第7節] で考えられている．

:::

## 数値実験：MSE の比較

ある Gauss 分布に従うデータを生成する：
$$
Y^j\iidsim\rN(x_0,\sigma^2),\qquad j\in[n],
$$
分散 $\sigma^2$ は既知として，位置母数 $x\in\R$ を推定する問題を考える．

事前分布を $\rN(0,\rho^2)$ とすると，定数の違いを除いて
\begin{align*}
    U(x)&=\frac{x^2}{2\rho^2}+\frac{1}{2\sigma^2}\sum_{j=1}^n(x-y^j)^2\\
    &=\frac{1}{n}\sum_{j=1}^n\paren{\frac{x^2}{2\rho^2}+\frac{n}{2\sigma^2}(x-y^j)^2}=:\frac{1}{n}\sum_{j=1}^nU^j(x)
\end{align*}
であるから，
\begin{align*}
    U'(x)&=\frac{x}{\rho^2}+\frac{1}{\sigma^2}\sum_{j=1}^n(x-y^j)\\
    &=\frac{x}{\rho^2}+\frac{n}{\sigma^2}(x-\ov{y}),
\end{align*}
$$
U''(x)=\frac{1}{\rho^2}+\frac{n}{\sigma^2}.
$$

従って，Zig-Zag 過程のイベントの強度関数は
\begin{align*}
    m(t)&=\Paren{\theta U'(x+\theta t)}_+\\
    &=\paren{\frac{\theta(x+\theta t)}{\rho^2}+\frac{\theta}{\sigma^2}\sum_{j=1}^n(x+\theta t-y^j)}_+\\
    &=\paren{\frac{\theta x}{\rho^2}+\frac{\theta}{\sigma^2}\sum_{j=1}^n(x-y^j)+t\paren{\frac{1}{\rho^2}+\frac{n}{\sigma^2}}}_+
\end{align*}
と表せ，これは１次関数 $(a+bt)_+$ の形であるから直接のシミュレーションが可能である．^[実装は `ZigZagBoomerang` パッケージの [`zigzagboom1d.jl`](https://github.com/mschauer/ZigZagBoomerang.jl/blob/master/src/zigzagboom1d.jl) を参考にした．]


In [ ]:
#| code-fold: true
#| output: false
#| code-summary: サブサンプリングなしの Zig-Zag 過程のシミュレーションをする関数 ZZ() を定義
using ZigZagBoomerang
using Distributions
using Random

λ(∇U, x, θ, F::ZigZag1d) = pos(θ*∇U(x)) # rate function on E
λ_bar(τ, a, b) = pos(a + b*τ)  # affine proxy

"""
`x`: current location, `θ`: current velocity, `t`: current time,
"""
function move_forward(τ, t, x, θ, ::ZigZag1d)
    τ + t, x + θ*τ , θ
end

"""
    `∇U`: gradient of the negative log-density
    `(x,θ)`: initial state
    `T`: Time Horizon    
    `a+bt`: computational bound for intensity m(t)

    `num`: ポアソン時刻に到着した回数
    `acc`: 受容回数．`acc/num` は acceptance rate
"""
function ZZ(∇U, x::Float64, θ::Float64, T::Float64, y, Flow::ZigZagBoomerang.ContinuousDynamics; rng=Random.GLOBAL_RNG, ab=ab_ZZ)
    t = zero(T)
    Ξ = [(t, x, θ)]
    num = acc = 0
    epoch_list = [num]
    a, b = ab(x, θ, Flow)
    t′ =  t + poisson_time(a, b, rand())  # イベントは a,b が定める affine proxy に従って生成する

    while t < T
        τ = t′ - t
        t, x, θ = move_forward(τ, t, x, θ, Flow)
        l, lb = λ(∇U, x, θ, Flow), λ_bar(τ, a, b)  # λ が真のレート, λ_bar が affine proxy
        num += 1
        if rand()*lb < l
            acc += 1
            if l > lb + 0.01
                println(l-lb)
            end
            θ = -θ
            push!(Ξ, (t, x, θ))
            push!(epoch_list, num)
        end
        a, b = ab(x, θ, Flow)
        t′ = t + poisson_time(a, b, rand())
    end

    return Ξ, epoch_list, acc/num
end

In [ ]:
#| output: false
#| code-fold: true
#| code-summary: 今回の設定に応じたレート関数 (a+bt)+ を用意
pos(x) = max(zero(x), x)  # positive part
a(x, θ, ρ, σ, y) = θ * x / ρ^2 + (θ/σ^2) * sum(x .- y)
b(x, θ, ρ, σ, y) = ρ^(-2) + length(y)/σ^2

ρ, σ, x0, θ0 = 1.0, 1.0, 1.0, 1.0
n1, n2 = 100, 10^4
TrueDistribution = Normal(x0, σ)
y1 = rand(TrueDistribution, n1)
y2 = rand(TrueDistribution, n2)

# computational bounds for intensity m(t)
ab_ZZ_n1(x, θ, ::ZigZag1d) = (a(x, θ, ρ, σ, y1), b(x, θ, ρ, σ, y1))
ab_ZZ_n2(x, θ, ::ZigZag1d) = (a(x, θ, ρ, σ, y2), b(x, θ, ρ, σ, y2))

∇U1(x) = x/ρ^2 + (length(y1)/σ^2) * (x - mean(y1)) 
∇U2(x) = x/ρ^2 + (length(y2)/σ^2) * (x - mean(y2)) 

# T = 2500.0
# trace_ZZ1, epochs_ZZ1, acc_ZZ1 = ZZ(∇U1, x0, θ0, T, ZigZag1d(); ab=ab_ZZ_n1)
# trace_ZZ2, num_ZZ2, acc_ZZ2 = ZZ(∇U2, x0, θ0, T, ZigZag1d(); ab=ab_ZZ_n2)
# dt = 0.01
# traj_ZZ1 = discretize(trace_ZZ1, ZigZag1d(), dt)
# traj_ZZ2 = discretize(trace_ZZ2, ZigZag1d(), dt)

In [ ]:
#| output: false
#| code-fold: true
#| code-summary: N 回 ZZ() を実行して，その事後平均の MSE を計算する関数 experiment() を定義
function SquaredError(sample::Vector{Float64}, y)
    True_Posterior_Mean = sum(y) / (length(y) + 1)
    return (mean(sample) - True_Posterior_Mean)^2
end

"""
    epoch_list: 注目するエポック数のリスト
    N: 実験回数
"""
function experiment(epoch_list, T, dt, N, ∇U, x0, θ0, y, Sampler; ab=ab_ZZ_n1)
    SE_sum = zero(epoch_list)
    acc_list = []
    for _ in 1:N
        trace_ZZ1, epochs_ZZ1, acc_ZZ1 = Sampler(∇U, x0, θ0, T, y, ZigZag1d(); ab=ab)
        push!(acc_list, acc_ZZ1)
        traj_ZZ1 = discretize(trace_ZZ1, ZigZag1d(), dt)
        SE_list = []
        for T in epoch_list
            epoch = findfirst(x -> x > T, epochs_ZZ1) - 1
            t = findfirst(x -> x > trace_ZZ1[epoch][1], traj_ZZ1.t) - 1
            SE = SquaredError(traj_ZZ1.x[1:t], y)
            push!(SE_list, SE)
        end
        SE_sum += SE_list
    end
    return SE_sum ./ N, mean(acc_list)
end

In [ ]:
#| code-fold: true
#| code-summary: 実験の実行
using Plots

T = 3000.0
epoch_list = [10.0, 100.0, 1000.0, 10000.0]
dt = 0.01
N = 11

MSE_ZZ1, acc = experiment(epoch_list, T, dt, N, ∇U1, x0, θ0, y1, ZZ; ab=ab_ZZ_n1)
p = plot(#epoch_list, MSE_ZZ1,
    xscale=:log10,
    yscale=:log10,
    xlabel="epochs",
    ylabel="MSE"
    # ,background_color = "#F0F1EB"
    )
scatter!(p, epoch_list, MSE_ZZ1,
    marker=:circle,
    markersize=5,
    markeralpha=0.6,
    color="#78C2AD",
    label=nothing
    )

using GLM, DataFrames
df = DataFrame(X = log10.(epoch_list), Y = log10.(MSE_ZZ1))
model = lm(@formula(Y ~ X), df)
X_pred = range(minimum(df.X), maximum(df.X), length=100)
Y_pred = predict(model, DataFrame(X = X_pred))
plot!(p, 10 .^ X_pred, 10 .^ Y_pred,
    line=:solid,
    linewidth=2,
    color="#78C2AD",
    label="ZZ"
    )

# display(p)

println("Average acceptance rate: $acc")

より，たしかに剪定なしの正確なシミュレーションができている．

一方で，
$$
U^j(x):=\frac{x^2}{2\rho^2}+\frac{n}{2\sigma^2}(x-y^j)^2,
$$
$$
\lambda^j(x,\theta):=\Paren{\theta(U^j)'(x)}_+
$$
としてサブサンプリングを取り入れることを考えるが，これを同じ $(a+bt)_+$ ではバウンド出来ない：


In [ ]:
#| code-fold: true
#| code-summary: ZZ-SS (ZigZag with Subsampling) の定義
#| output: false

λj(j,x,θ,y) = pos(θ * (x/ρ^2 + length(y)/σ^2 * (x - y[j])))

function ZZ_SS(∇U, x::Float64, θ::Float64, T::Float64, y, Flow::ZigZagBoomerang.ContinuousDynamics; rng=Random.GLOBAL_RNG, ab=ab_ZZ)
    t = zero(T)
    Ξ = [(t, x, θ)]
    num = acc = 0
    epoch_list = [num]
    a, b = ab(x, θ, Flow)
    t′ =  t + poisson_time(a, b, rand())  # イベントは a,b が定める affine proxy に従って生成する

    while t < T
        τ = t′ - t
        t, x, θ = move_forward(τ, t, x, θ, Flow)
        j = rand(1:length(y))
        l, lb = λj(j, x, θ, y), λ_bar(τ, a, b)  # λ が真のレート, λ_bar が affine proxy
        num += 1
        if rand()*lb < l
            if l > lb + 0.01
                # println(l-lb)
                acc += 1  #  overflow を数えるように変更済み！注意！
            end
            θ = -θ
            push!(Ξ, (t, x, θ))
            push!(epoch_list, num)
        end
        a, b = ab(x, θ, Flow)
        t′ = t + poisson_time(a, b, rand())
    end

    return Ξ, epoch_list, acc/num
end

In [ ]:
#| code-fold: true
#| code-summary: 実験の実行
using LaTeXStrings

MSE_ZZ_SS, acc = experiment(epoch_list, T, dt, N, ∇U1, x0, θ0, y1, ZZ_SS; ab=ab_ZZ_n1)
println(L"上界 $(a+bt)_+$ を超えてしまう平均的割合: ", "$acc")

しかし，ZZ-CV アルゴリズムではこのようなことは起こらない．実際，次の等式が成り立つ：
$$
U'(x_*)+(U^j)'(x)-(U^j)'(x_*)=U'(x),\qquad x,x_*\in\R.
$$

このモデルにおける MAP 推定量は
$$
\wh{x}:=\frac{\ov{y}}{1+\frac{\sigma^2}{n\rho^2}}
$$
である．


In [ ]:
#| code-fold: true
#| code-summary: ZZ-CV (ZigZag with Control Variates) の定義
#| output: false

x_star = mean(y1) / (1 + σ^2/(length(y1) * ρ^2))

C(ρ, σ, y) = ρ^(-2) + length(y)/σ^2
a(x, θ, ρ, σ, y) = pos(θ*∇U1(x_star)) + C(ρ, σ, y) * abs(x - x_star)
b(x, θ, ρ, σ, y) = C(ρ, σ, y)

# New Computational Bounds for ZZ-CV
ab_ZZ_CV(x, θ, ::ZigZag1d) = (a(x, θ, ρ, σ, y1), b(x, θ, ρ, σ, y1))

function ZZ_CV(∇U, x::Float64, θ::Float64, T::Float64, y, Flow::ZigZagBoomerang.ContinuousDynamics; rng=Random.GLOBAL_RNG, ab=ab_ZZ_CV)
    t = zero(T)
    Ξ = [(t, x, θ)]
    num = acc = 0
    epoch_list = [num]
    a, b = ab(x, θ, Flow)
    t′ =  t + poisson_time(a, b, rand())  # イベントは a,b が定める affine proxy に従って生成する

    while t < T
        τ = t′ - t
        t, x, θ = move_forward(τ, t, x, θ, Flow)
        # j = rand(1:length(y))  # 今回はたまたま要らない
        l, lb =λ(∇U, x, θ, Flow), λ_bar(τ, a, b)  # λ が真のレート, λ_bar が affine proxy
        num += 1
        if rand()*lb < l
            acc += 1
            if l > lb + 0.01
                println(l-lb)
            end
            θ = -θ
            push!(Ξ, (t, x, θ))
            push!(epoch_list, num)
        end
        a, b = ab(x, θ, Flow)
        t′ = t + poisson_time(a, b, rand())
    end

    return Ξ, epoch_list, acc/num
end

In [ ]:
#| code-fold: true
#| code-summary: 実験の実行

MSE_ZZ_CV, acc = experiment(epoch_list, T, dt, N, ∇U1, x0, θ0, y1, ZZ_CV; ab=ab_ZZ_CV)

q = scatter(p, epoch_list, MSE_ZZ_CV,
    marker=:circle,
    markersize=5,
    markeralpha=0.6,
    color="#E95420",
    label=nothing
    )

df = DataFrame(X = log10.(epoch_list), Y = log10.(MSE_ZZ_CV))
model = lm(@formula(Y ~ X), df)
X_pred = range(minimum(df.X), maximum(df.X), length=100)
Y_pred = predict(model, DataFrame(X = X_pred))
plot!(q, 10 .^ X_pred, 10 .^ Y_pred,
    line=:dash,
    linewidth=2,
    color="#E95420",
    label="ZZ-CV (without amendment)"
    )

display(q)

In [ ]:
#| output: false
#| echo: false
savefig(q, "MeanOfGaussian_Erronous.svg")

一見すると ZZ-CV が負けているように見える．しかし，点線で描いているのは，横軸が epoch であることを正しく考慮していない間違ったプロットであるためである．

[@Bierkens+2019 p.1310] において epoch とは，計算量の１単位分としており，ZZ における１回の到着時刻のシミュレーションは，ZZ-CV の $n$ 回分に当たる．これを考慮に入れてプロットし直すと次の通りになる：


In [ ]:
#| code-fold: true
#| code-summary: 実験の実行
T_SuperEfficient = 300000.0
epoch_list_SuperEfficient = [1000.0, 10000.0, 100000.0, 1000000.0]

@time MSE_ZZ_CV, acc = experiment(epoch_list_SuperEfficient, T_SuperEfficient, dt, N, ∇U1, x0, θ0, y1, ZZ_CV; ab=ab_ZZ_CV)

scatter!(p, epoch_list, MSE_ZZ_CV,
    marker=:circle,
    markersize=5,
    markeralpha=0.6,
    color="#E95420",
    label=nothing
    )

df = DataFrame(X = log10.(epoch_list), Y = log10.(MSE_ZZ_CV))
model = lm(@formula(Y ~ X), df)
X_pred = range(minimum(df.X), maximum(df.X), length=100)
Y_pred = predict(model, DataFrame(X = X_pred))
plot!(p, 10 .^ X_pred, 10 .^ Y_pred,
    line=:solid,
    linewidth=2,
    color="#E95420",
    label="ZZ-CV"
    )

display(p)

In [ ]:
#| output: false
#| echo: false
savefig(p, "MeanOfGaussian.svg")

これは換言すれば横軸が「ズルい」ということでもあるが，同時に $n\to\infty$ の極限では，圧倒的に ZZ-CV が効率的になるということでもある．^[ただし，例えば今回も計算時間で言えば長くなっていることに注意．]

## MALA との比較


In [ ]:
#| code-fold: true
#| code-summary: MALA のセットアップ
#| output: false
using AdvancedHMC, AdvancedMH, ForwardDiff
using LogDensityProblems
using LogDensityProblemsAD
using StructArrays
using LinearAlgebra

struct LogTargetDensity
    y::Vector{Float64}
end

function U(i, x, y)
    x[1] * x[1] / (2 * ρ * ρ) + length(y) * (x[1] - y[i]) * (x[1] - y[i]) / (2 * σ * σ)  # 自動微分のために x は長さ1のベクトルと扱う必要がある
end

function U(x, y)
    vec = [U(i, x, y) for i in 1:length(y)]
    return mean(vec)
end

LogDensityProblems.logdensity(p::LogTargetDensity, x) = U(x, p.y)
LogDensityProblems.dimension(p::LogTargetDensity) = 1
LogDensityProblems.capabilities(::Type{LogTargetDensity}) = LogDensityProblems.LogDensityOrder{0}()

model_with_ad = LogDensityProblemsAD.ADgradient(Val(:ForwardDiff), LogTargetDensity(y1))

# σ² = 0.1 # ほぼ横ばい
# σ² = 0.5 # １回小さいエポック10で効率勝った＋全く横ばいになった
σ² = 0.2  # すごく良い感じ
spl = MALA(x -> MvNormal((σ² / 2) .* x, σ² * I))

In [ ]:
#| code-fold: true
#| output: false
#| code-summary: 実験の実行
function experiment_MALA(epoch_list, N, y)
    SE_sum = zero(epoch_list)
    for _ in 1:N
        chain = sample(model_with_ad, spl, Int64(epoch_list[end]); initial_params=[x0], chain_type=StructArray, param_names=["x"], stats=true)
        traj_MALA = Vector{Float64}(chain.x)
        SE_list = []
        for T in epoch_list
            SE = SquaredError(traj_MALA[1:T], y)
            push!(SE_list, SE)
        end
        SE_sum += SE_list
    end
    return SE_sum ./ N
end

MSE_MALA = experiment_MALA(Vector{Int64}(epoch_list), N, y1)

In [ ]:
#| code-fold: true
#| code-summary: 結果のプロット
scatter!(p, epoch_list, MSE_MALA,
    marker=:circle,
    markersize=5,
    markeralpha=0.6,
    color="blue",
    label=nothing
    )

df = DataFrame(X = log10.(epoch_list), Y = log10.(MSE_MALA))
model = lm(@formula(Y ~ X), df)
X_pred = range(minimum(df.X), maximum(df.X), length=100)
Y_pred = predict(model, DataFrame(X = X_pred))
plot!(p, 10 .^ X_pred, 10 .^ Y_pred,
    line=:solid,
    linewidth=2,
    color="blue",
    label="MALA"
    )

display(p)

In [ ]:
#| output: false
#| echo: false
savefig(p, "MeanOfGaussian_addedMALA.svg")

## 非一様な部分サンプリング

当然，必ずしも一様な分解
$$
U(x)=\frac{1}{n}\sum_{j=1}^nU^j(x)
$$
に基づいた一様なサブサンプリング $K\sim\rU([n])$ を行う必要はない．

剪定の手続きを棄却法とみると，重点サンプリングのアイデアを導入することで制御変数に依らない分散低減が狙える [@Sen+2020 importance subsampling strategy]．

特に，比例的高次元極限や，不均衡データに対する[ロジスティック回帰](../Stat/Logistic.qmd)では，事後分布が十分な集中性を持たないために制御変数の方法 [-@sec-ZZ-CV] が十分な効率改善を示さないが，この重点サブサンプリングによれば効率の改善が見込める．

詳しくは，次稿参照：


```{=html}
<div class="article-card-container">
    <div class="article-card">
        <a href="https://162348.github.io/posts/2024/Stat/Logistic2.html" target="_blank">
            <img src="https://162348.github.io/posts/2024/Stat/Logistic2.svg" alt="Article Image" class="article-image">
            <div class="article-content">
                <h3 class="article-title">大規模な不均衡データに対するロジスティック回帰（後編）</h3>
                <p class="article-description">離散時間 MCMC から連続時間 MCMC へ</p>
            </div>
        </a>
    </div>
</div>
```